#Import library

In [8]:
import os
import pandas as pd              # Library used for working with data sets and perform data analysis.
import numpy as np                  # To perform mathematical operasion and statistics
from datetime import datetime       # Used to perform dates and time manipulation


#Import dataset

In [29]:
path = r"C:\Users\prada\Documents\Latihan\Latihan_NLP\data\scrapped_data2.csv"
df = pd.read_csv(path, encoding='latin1')
df.head()

,userName,score,at,content
0,Achsan Ansori,2,10/21/2023 15:36,"21/10/2023, saya update ke versi 7.2.0 perform..."
1,Yanuar Syam,1,10/27/2023 10:07,Setelah update malah jadi force close terus. S...
2,Farhan Alfariqi,3,10/29/2023 18:38,"Menurut saya, aplikasi sekarang sangat lambat...."
3,Yeshua's Princess,1,10/28/2023 16:06,"Perasaan 17 Agustus udah lama lewat, koq tampi..."
4,Fernando solafide Napitupulu,2,10/23/2023 22:23,Paket kuota Apps katanya akan aktif jika paket...


#Data Preparation

In [30]:
df_cp = df

##Data cleanining

memberikan label jika bintang <= 2 maka negatif jika > 3 maka positif, bintang 3 tidak dipakai

In [31]:
#membuat kondisi
conditions = [
    (df_cp['score'] > 3),
    (df_cp['score'] < 3)
]
#membuat kategori
categories = ['positive', 'negative']
#memberikan kategori ke data
df_cp['label'] = np.select(conditions, categories, default='Unknown')
df_cp.head()

,userName,score,at,content,label
0,Achsan Ansori,2,10/21/2023 15:36,"21/10/2023, saya update ke versi 7.2.0 perform...",negative
1,Yanuar Syam,1,10/27/2023 10:07,Setelah update malah jadi force close terus. S...,negative
2,Farhan Alfariqi,3,10/29/2023 18:38,"Menurut saya, aplikasi sekarang sangat lambat....",Unknown
3,Yeshua's Princess,1,10/28/2023 16:06,"Perasaan 17 Agustus udah lama lewat, koq tampi...",negative
4,Fernando solafide Napitupulu,2,10/23/2023 22:23,Paket kuota Apps katanya akan aktif jika paket...,negative


menghapus score bintang 3

In [32]:
df_cp = df_cp[df_cp['score'] != 3]
df_cp.head()

,userName,score,at,content,label
0,Achsan Ansori,2,10/21/2023 15:36,"21/10/2023, saya update ke versi 7.2.0 perform...",negative
1,Yanuar Syam,1,10/27/2023 10:07,Setelah update malah jadi force close terus. S...,negative
3,Yeshua's Princess,1,10/28/2023 16:06,"Perasaan 17 Agustus udah lama lewat, koq tampi...",negative
4,Fernando solafide Napitupulu,2,10/23/2023 22:23,Paket kuota Apps katanya akan aktif jika paket...,negative
5,Anwar,1,10/21/2023 22:25,"1. Setelah update, aplikasi semakin berat. 2. ...",negative


menghapus kolom username, score, dan at

In [33]:
df_edit = df_cp[['content', 'label']]

In [34]:
df_edit.head()

,content,label
0,"21/10/2023, saya update ke versi 7.2.0 perform...",negative
1,Setelah update malah jadi force close terus. S...,negative
3,"Perasaan 17 Agustus udah lama lewat, koq tampi...",negative
4,Paket kuota Apps katanya akan aktif jika paket...,negative
5,"1. Setelah update, aplikasi semakin berat. 2. ...",negative


In [35]:
df_edit['message_len'] = df_edit['content'].apply(lambda x: len(x.split(' ')))
df_edit.head()

C:\Users\prada\AppData\Local\Temp\ipykernel_2104\2802504994.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edit['message_len'] = df_edit['content'].apply(lambda x: len(x.split(' ')))


,content,label,message_len
0,"21/10/2023, saya update ke versi 7.2.0 perform...",negative,70
1,Setelah update malah jadi force close terus. S...,negative,49
3,"Perasaan 17 Agustus udah lama lewat, koq tampi...",negative,48
4,Paket kuota Apps katanya akan aktif jika paket...,negative,38
5,"1. Setelah update, aplikasi semakin berat. 2. ...",negative,75


menglihat keseimbangan data

In [36]:
balance_counts = df_edit.groupby('label')['label'].agg('count').values
balance_counts

array([841,  88], dtype=int64)

#membersihkan data dari karakter yang tidak digunakan

In [37]:
import re
import string

def clean_text(text):
    '''Make text lowercase, remove text in square brackets,remove links,remove punctuation
    and remove words containing numbers.'''
    text = str(text).lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

In [38]:
df_edit['message_clean'] = df_edit['content'].apply(clean_text)
df_edit.head()

C:\Users\prada\AppData\Local\Temp\ipykernel_2104\900754444.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_edit['message_clean'] = df_edit['content'].apply(clean_text)


,content,label,message_len,message_clean
0,"21/10/2023, saya update ke versi 7.2.0 perform...",negative,70,saya update ke versi performa aplikasi masih...
1,Setelah update malah jadi force close terus. S...,negative,49,setelah update malah jadi force close terus se...
3,"Perasaan 17 Agustus udah lama lewat, koq tampi...",negative,48,perasaan agustus udah lama lewat koq tampilan...
4,Paket kuota Apps katanya akan aktif jika paket...,negative,38,paket kuota apps katanya akan aktif jika paket...
5,"1. Setelah update, aplikasi semakin berat. 2. ...",negative,75,setelah update aplikasi semakin berat progra...
